In [1]:
import pandas as pd
import pandas as pd
import numpy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


In [2]:
df=pd.read_csv('data.csv')
df2 = pd.read_csv('cleaned.csv')

In [3]:
df.columns

Index(['user_id', 'user_black_draw_rate', 'user_black_game_count',
       'user_black_lose_rate', 'user_black_win_rate', 'user_blitz_rating',
       'user_draw_rate', 'user_game_count',
       'user_highest_blitz_opponent_rating', 'user_lose_rate',
       'user_loss_streak_current', 'user_loss_streak_max',
       'user_rating_deviation', 'user_second_highest_blitz_opponent_rating',
       'user_third_highest_blitz_opponent_rating', 'user_total_games_played',
       'user_white_draw_rate', 'user_white_game_count', 'user_white_lose_rate',
       'user_white_win_rate', 'user_win_rate', 'user_win_streak_current',
       'user_win_streak_max', 'user_op', 'op_black_draw_rate',
       'op_black_game_count', 'op_black_lose_rate', 'op_black_win_rate',
       'op_blitz_rating', 'op_draw_rate', 'op_game_count',
       'op_highest_blitz_opponent_rating', 'op_lose_rate',
       'op_loss_streak_current', 'op_loss_streak_max', 'op_rating_deviation',
       'op_second_highest_blitz_opponent_rating',
 

In [4]:
df2.columns

Index(['user_black_game_count', 'user_black_draw_rate', 'user_black_win_rate',
       'user_blitz_rating', 'user_draw_rate', 'user_game_count',
       'user_highest_blitz_opponent_rating', 'user_win_rate',
       'user_rating_deviation', 'user_total_games_played',
       'user_white_draw_rate', 'user_white_game_count', 'user_white_win_rate',
       'op_black_game_count', 'op_black_draw_rate', 'op_black_win_rate',
       'op_blitz_rating', 'op_draw_rate', 'op_game_count',
       'op_highest_blitz_opponent_rating', 'op_win_rate',
       'op_rating_deviation', 'op_total_games_played', 'op_white_draw_rate',
       'op_white_game_count', 'op_white_win_rate', 'game_result', 'user_id',
       'user_op'],
      dtype='object')

In [5]:
columns_not_in_df2 = [col for col in df.columns if col not in df2.columns]

columns_not_in_df2

['user_black_lose_rate',
 'user_lose_rate',
 'user_loss_streak_current',
 'user_loss_streak_max',
 'user_second_highest_blitz_opponent_rating',
 'user_third_highest_blitz_opponent_rating',
 'user_white_lose_rate',
 'user_win_streak_current',
 'user_win_streak_max',
 'op_black_lose_rate',
 'op_lose_rate',
 'op_loss_streak_current',
 'op_loss_streak_max',
 'op_second_highest_blitz_opponent_rating',
 'op_third_highest_blitz_opponent_rating',
 'op_white_lose_rate',
 'op_win_streak_current',
 'op_win_streak_max']

In [6]:
# #lose and win rate cross data for each player
# df['user_performace'] = df['user_win_rate'] * (df['op_lose_rate'])
# df['op_performance'] = df['op_win_rate'] * (df['user_lose_rate'])

In [7]:
#average highest beaten for each player
df['op_average_best_win'] = df[['op_second_highest_blitz_opponent_rating',
 'op_third_highest_blitz_opponent_rating', 'op_highest_blitz_opponent_rating']].mean(axis=1)
df['user_average_best_win'] = df[['user_second_highest_blitz_opponent_rating', 'user_third_highest_blitz_opponent_rating', 'user_highest_blitz_opponent_rating']].mean(axis=1)


In [8]:
#highest opponent beaten difference between two players
df['best_diff'] = df['op_highest_blitz_opponent_rating'] - df['user_highest_blitz_opponent_rating']

In [9]:
#rating difference between two players
df['rating_diff'] = df['user_blitz_rating'] - df['op_blitz_rating']

In [10]:
#draw related feature
# df['draw_rate'] = df['user_draw_rate'] +df['op_draw_rate']
df['combined_draw4'] = (df['user_white_draw_rate'] + df['op_black_draw_rate'] + df['user_black_draw_rate'] + df['op_white_draw_rate'])/4

In [11]:
# #compare the number of recent games between two users
df['game_count_ratio'] = df['user_game_count'] / df['op_game_count']

In [12]:
#color specific performances
df['user_white_performance'] = df['user_white_win_rate'] * (df['op_black_lose_rate'])
df['user_black_performance'] = df['user_black_win_rate'] * (df['op_white_lose_rate'])
df['op_white_performance'] = df['op_white_win_rate'] * (df['user_black_lose_rate'])
df['op_black_performance'] = df['op_black_win_rate'] * (df['user_white_lose_rate'])

In [13]:
to_remove2 = ['user_loss_streak_current', 'user_loss_streak_max', 'user_second_highest_blitz_opponent_rating', 'user_third_highest_blitz_opponent_rating', 'user_highest_blitz_opponent_rating', 'user_win_streak_current',
 'user_win_streak_max', 'op_loss_streak_current',
 'op_loss_streak_max',
 'op_second_highest_blitz_opponent_rating',
 'op_third_highest_blitz_opponent_rating', 'op_highest_blitz_opponent_rating',  'op_win_streak_current',
 'op_win_streak_max']

In [14]:
df = df.drop(columns=to_remove2)

In [15]:


data = df

# our features
X = data.drop(columns=['game_result', 'user_id', 'user_op'])

#our label
y = data['game_result']   # the last column

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the KNN model
knn = KNeighborsClassifier()

# Set up hyperparameter grid for tuning
param_grid = {
    'n_neighbors': [10, 50, 100],          # Two choices for number of neighbors
    'weights': ['uniform', 'distance']  # Two choices for weighting
}


# Perform grid search with cross-validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_knn = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Best Parameters:", grid_search.best_params_)
print("Accuracy with Best Parameters:", accuracy)

Best Parameters: {'n_neighbors': 100, 'weights': 'uniform'}
Accuracy with Best Parameters: 0.5324881141045958


In [16]:

data = df

# our features
X = data.drop(columns=['game_result', 'user_id', 'user_op'])

#our label
y = data['game_result']   # the last column

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Initialize the KNN model
rf = RandomForestClassifier(random_state=42)

# Set up hyperparameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [5, 10, 20]       # Maximum depth of the trees
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_knn = grid_search.best_estimator_

# Make predictions with the best model
y_pred = best_knn.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Best Parameters:", grid_search.best_params_)
print("Accuracy with Best Parameters:", accuracy)

Best Parameters: {'max_depth': 5, 'n_estimators': 50}
Accuracy with Best Parameters: 0.5578446909667195


c:\Users\tiazh\OneDrive\CS\CS506\cs506_final_project\.venv\Lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [17]:
from xgboost import XGBClassifier

data = df

# our features
X = data.drop(columns=['game_result', 'user_id', 'user_op'])

#our label
y = data['game_result']   # the last column
y = y.map({-1: 0, 0: 1, 1: 2})

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


param_grid = {
    'learning_rate': [0.02, 0.05, 0.1],       # Step size shrinkage
    'max_depth': [4, 6],                # Maximum tree depth
    'n_estimators': [100, 300],         # Number of trees
    'subsample': [0.8, 1.0],            # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 1.0]      # Subsample ratio of columns when constructing each tree
}
model = XGBClassifier(random_state=42)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,               # 5-fold cross-validation
    verbose=1,          # Verbosity level
    n_jobs=1           # Utilize all available CPU cores
)

grid_search.fit(X_train, y_train)

# model.fit(X_train, y_train)
# Set up hyperparameter grid for tuning
# param_grid = {
#     'n_estimators': [50, 100],  # Number of trees in the forest
#     'max_depth': [10, 20]       # Maximum depth of the trees
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # Get the best model
# best_knn = grid_search.best_estimator_

# # Make predictions with the best model
# y_pred = best_knn.predict(X_test)

best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Best Parameters:", grid_search.best_params_)
print("Accuracy with Best Parameters:", accuracy)

Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.02, 'max_depth': 6, 'n_estimators': 100, 'subsample': 0.8}
Accuracy with Best Parameters: 0.5686015831134564


In [18]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, InputLayer
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers, losses, metrics

# Load data
data = df

# Map the labels from [-1, 0, 1] to [0, 1, 2]
label_mapping = {-1: 0, 0: 1, 1: 2}
data['game_result'] = data['game_result'].map(label_mapping)

# Define features and labels
X = data.drop(columns=['game_result', 'user_id', 'user_op'])
y = data['game_result']  # No one-hot encoding required

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Apply Target Encoding
target_encoder = TargetEncoder(cols=categorical_cols)
X_train_encoded = target_encoder.fit_transform(X_train, y_train)
X_test_encoded = target_encoder.transform(X_test)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

# Define the neural network model
nn_model = Sequential()

# Input layer
nn_model.add(InputLayer(input_shape=(X_train_scaled.shape[1],), name='input'))

# Hidden layers
nn_model.add(Dense(units=128, activation='relu', name='hl_1'))
nn_model.add(Dropout(0.2))

nn_model.add(Dense(units=64, activation='relu', name='hl_2'))
nn_model.add(Dropout(0.2))

nn_model.add(Dense(units=32, activation='relu', name='hl_3'))
nn_model.add(Dropout(0.2))

# Output layer
nn_model.add(Dense(units=3, activation='softmax', name='output'))  # 3 units for 3 classes

# Compile the model with a lower learning rate
nn_model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy()]
)

# Custom callback for logging progress
class ProgBarLoggerNEpochs(callbacks.Callback):
    def __init__(self, num_epochs: int, every_n: int = 50):
        self.num_epochs = num_epochs
        self.every_n = every_n

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.every_n == 0:
            log_message = f"Epoch [{epoch + 1}/{self.num_epochs}]"
            log_details = ", ".join([f"{k.capitalize()}: {v:.4f}" for k, v in logs.items()])
            print(f"{log_message}, {log_details}")

# Early stopping callback
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
t0 = time.time()  # Start time
num_epochs = 500

history = nn_model.fit(
    X_train_scaled,
    y_train,
    epochs=num_epochs,
    validation_split=0.2,
    verbose=0,  # Disable default progress bar
    callbacks=[ProgBarLoggerNEpochs(num_epochs, every_n=50), early_stop],
)
t1 = time.time()  # End time

print(f"Elapsed time: {t1 - t0:.2f}s")

# Evaluate the model on the test set
loss, accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Predictions
probability_predictions = nn_model.predict(X_test_scaled)
predicted_classes = probability_predictions.argmax(axis=1)

# Evaluate performance using classification metrics
print(confusion_matrix(y_test, predicted_classes))
print(classification_report(y_test, predicted_classes))

c:\Users\tiazh\OneDrive\CS\CS506\cs506_final_project\.venv\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Elapsed time: 4.85s
Test Loss: 0.8155, Test Accuracy: 0.5246
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
[[264   0  63]
 [ 15   0   6]
 [216   0  67]]
              precision    recall  f1-score   support

           0       0.53      0.81      0.64       327
           1       0.00      0.00      0.00        21
           2       0.49      0.24      0.32       283

    accuracy                           0.52       631
   macro avg       0.34      0.35      0.32       631
weighted avg       0.50      0.52      0.48       631



c:\Users\tiazh\OneDrive\CS\CS506\cs506_final_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tiazh\OneDrive\CS\CS506\cs506_final_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\tiazh\OneDrive\CS\CS506\cs506_final_project\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf

df['user_performace'] = df['user_win_rate'] * (df['op_lose_rate'])
df['op_performance'] = df['op_win_rate'] * (df['user_lose_rate'])
remove this for:
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 100, 'subsample': 1.0}
Accuracy with Best Parameters: 0.5699208443271768
Best Parameters: {'max_depth': 10, 'n_estimators': 50}
Accuracy with Best Parameters: 0.5641838351822503